In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

$nn.Linear$

In [2]:
x = torch.randn(1,784)

In [3]:
layer1 = nn.Linear(784,200)
layer2 = nn.Linear(200,200)
layer3 = nn.Linear(200,10)

In [4]:
x = layer1(x)
x.shape

torch.Size([1, 200])

In [5]:
x = layer2(x)
x.shape

torch.Size([1, 200])

In [6]:
x = layer3(x)
x.shape

torch.Size([1, 10])

$relu?$ 

In [7]:
x = torch.randn(1,784)

In [8]:
x = layer1(x)

In [9]:
x = F.relu(x,inplace = True)
x.shape

torch.Size([1, 200])

In [10]:
x = layer3(x)
X = F.relu(x,inplace = True)
x.shape

torch.Size([1, 10])

### $example$

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.ReLU(inplace = True),
            nn.Linear(200,200),
            nn.ReLU(inplace = True),
            nn.Linear(200,10),
            nn.ReLU(inplace = True)
        )
        
    def forward(self,x):
        x = self.model(x)
        
        return x

$Train$

In [12]:
epochs = 3
learning_rate = 1e-2
batch_size = 64

In [13]:
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),                       # 数据类型转化
                torchvision.transforms.Normalize((0.1307, ), (0.3081, )) # 数据归一化处理
    ])), batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data/',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307, ), (0.3081, ))
    ])),batch_size=batch_size,shuffle=False)

In [14]:
net = MLP()
optimizer = optim.SGD(net.parameters(),lr = learning_rate)
criteon = nn.CrossEntropyLoss()

for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        data = data.view(-1,28*28)
        logits = net(data)
        loss= criteon(logits,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.311191
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.105377
Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.340355
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.941758
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.578893
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.587863
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.312985
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.451108
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.258539
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.489289

Test set: Average loss: 0.0053, Accuracy: 9020/10000 (90%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.299722
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.331713
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.341233
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.355909
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.236620
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.322451
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.385833
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.192465
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.311206
T